<a href="https://colab.research.google.com/github/gupta24789/named-entity-recognition/blob/main/ner_cnnchar_lstm_crf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !wget https://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip -d embeddings/

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
import random
import itertools
import pandas as pd
import numpy as np


from pathlib import Path
from pprint import pprint


import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from torch import optim
import torch.autograd as autograd
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from seqeval import metrics

## Download Data

In [ ]:
# Path('data/train').mkdir(parents = True, exist_ok= True)
# Path('data/val').mkdir(parents = True, exist_ok= True)
# Path('data/test').mkdir(parents = True, exist_ok= True)

# os.system("cd data/train && wget https://raw.githubusercontent.com/gupta24789/named-entity-recognition/main/data/train/sentences.txt")
# os.system("cd data/train && wget https://raw.githubusercontent.com/gupta24789/named-entity-recognition/main/data/train/labels.txt")
# os.system("cd data/val && wget https://raw.githubusercontent.com/gupta24789/named-entity-recognition/main/data/val/sentences.txt")
# os.system("cd data/val && wget https://raw.githubusercontent.com/gupta24789/named-entity-recognition/main/data/val/labels.txt")
# os.system("cd data/test && wget https://raw.githubusercontent.com/gupta24789/named-entity-recognition/main/data/test/sentences.txt")
# os.system("cd data/test && wget https://raw.githubusercontent.com/gupta24789/named-entity-recognition/main/data/test/labels.txt")

## Set Seed

In [ ]:
seed = 121
random.seed(seed)
torch.manual_seed(seed)
pl.seed_everything(seed)

Seed set to 121


121

## Load Data

In [ ]:
## train
train_sents = open("data/train/sentences.txt","r").readlines()
train_tags = open("data/train/labels.txt","r").readlines()
## val
val_sents = open("data/val/sentences.txt","r").readlines()
val_tags = open("data/val/labels.txt","r").readlines()
## test
test_sents = open("data/test/sentences.txt","r").readlines()
test_tags = open("data/test/labels.txt","r").readlines()

In [ ]:
train_sents[:2]

['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .\n',
 'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "\n']

In [ ]:
train_tags[:2]

['O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O\n',
 'O O O O O O O O O O O O O O O O O O B-per O O O O O O O O O O O\n']

In [ ]:
X_train = [sent.strip().split(" ") for sent in train_sents]
y_train = [tag.strip().split(" ") for tag in train_tags]

X_val = [sent.strip().split(" ") for sent in val_sents]
y_val = [tag.strip().split(" ") for tag in val_tags]

X_test = [sent.strip().split(" ") for sent in test_sents]
y_test = [tag.strip().split(" ") for tag in test_tags]

In [ ]:
pprint((X_train[:2]), compact=True)

[['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London',
  'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the',
  'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.'],
 ['Families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined',
  'the', 'protesters', 'who', 'carried', 'banners', 'with', 'such', 'slogans',
  'as', '"', 'Bush', 'Number', 'One', 'Terrorist', '"', 'and', '"', 'Stop',
  'the', 'Bombings', '.', '"']]


In [ ]:
pprint(y_train[:1], compact=True)

[['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O',
  'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']]


## Create Vocab

In [ ]:
## char vocab
special_words = ['__PAD__','__UNK__']
char_vocab = list(set(itertools.chain.from_iterable(X_train + X_val + X_test)))
char_vocab = list(set(itertools.chain.from_iterable([list(w) for w in char_vocab])))
char_vocab = special_words + char_vocab
print(f'Char Vocab : {len(char_vocab)}')

char2idx = {char:i for i, char in enumerate(char_vocab)}
idx2char = {i:char for char,i in char2idx.items()}

CHAR_UNK_ID = char2idx['__UNK__']
CHAR_PAD_ID = char2idx['__PAD__']

Char Vocab : 100


In [ ]:
special_words = ['__PAD__','__UNK__']
vocab = list(set(itertools.chain.from_iterable(X_train + X_val + X_test)))
vocab = special_words + vocab
word2idx = {w:i for i,w in enumerate(vocab)}
idx2word = {i:w for w,i in word2idx.items()}

## TAGS
tags = list(set(itertools.chain.from_iterable(y_train)))
tags = ['__PAD__'] + tags
tag2idx = {w:i for i,w in enumerate(tags)}
idx2tag = {i:w for w,i in tag2idx.items()}


print(f"vocab size : {len(vocab)}")
print(f"tags : {len(tag2idx)}")
pprint(tag2idx, compact=True)

PAD_ID = word2idx['__PAD__']
UNK_ID = word2idx['__UNK__']

print(f"PAD ID : {PAD_ID}")

vocab size : 35180
tags : 18
{'B-art': 9,
 'B-eve': 11,
 'B-geo': 10,
 'B-gpe': 7,
 'B-nat': 8,
 'B-org': 16,
 'B-per': 6,
 'B-tim': 15,
 'I-art': 17,
 'I-eve': 12,
 'I-geo': 14,
 'I-gpe': 5,
 'I-nat': 4,
 'I-org': 1,
 'I-per': 2,
 'I-tim': 3,
 'O': 13,
 '__PAD__': 0}
PAD ID : 0


## Encode sent & tags

In [ ]:
def to_sent_number(sent_list):
    encoded = []
    for w in sent_list:
        encoded.append(word2idx.get(w, UNK_ID))
    return encoded


def to_tag_number(tag_list):
    encoded = []
    for tag in tag_list:
        encoded.append(tag2idx[tag])
    return encoded


In [ ]:
def to_char_number(sent_list):
    seq_char_list = []
    for word in sent_list:
        char_list = list(word)
        char_id = list()
        for char in char_list:
            char_id.append(char2idx.get(char, CHAR_UNK_ID))
        seq_char_list.append(char_id)
    return seq_char_list

In [ ]:
X_train_encoded = [to_sent_number(sent) for sent in X_train]
X_train_char_encoded = [to_char_number(sent) for sent in X_train]
y_train_encoded = [to_tag_number(tags) for tags in y_train]

X_val_encoded = [to_sent_number(sent) for sent in X_val]
X_val_char_encoded = [to_char_number(sent) for sent in X_val]
y_val_encoded = [to_tag_number(tags) for tags in y_val]

X_test_encoded = [to_sent_number(sent) for sent in X_test]
X_test_char_encoded = [to_char_number(sent) for sent in X_test]
y_test_encoded = [to_tag_number(tags) for tags in y_test]

In [ ]:
pprint(X_train_encoded[:2], compact=True)

[[26712, 34197, 33274, 11587, 21753, 25622, 30270, 6375, 29484, 8734, 34332,
  4607, 15481, 18687, 26749, 8734, 22975, 34197, 29366, 31635, 11812, 10530,
  30933, 32151],
 [23800, 34197, 30830, 19378, 4607, 8734, 35019, 23610, 8734, 31310, 2627,
  25256, 34769, 8757, 9212, 9076, 16113, 17026, 6945, 13859, 32377, 17675,
  17026, 18687, 17026, 16654, 8734, 17523, 32151, 17026]]


In [ ]:
pprint(X_train_char_encoded[:2], compact=True)

[[[59, 31, 79, 33, 32, 75, 2, 3, 32], [79, 14],
  [3, 20, 88, 79, 2, 32, 71, 21, 75, 71, 79, 21, 32], [31, 75, 81, 20],
  [88, 75, 21, 23, 31, 20, 3], [71, 31, 21, 79, 33, 30, 31],
  [40, 79, 2, 3, 79, 2], [71, 79], [61, 21, 79, 71, 20, 32, 71], [71, 31, 20],
  [91, 75, 21], [13, 2], [97, 21, 75, 48], [75, 2, 3], [3, 20, 88, 75, 2, 3],
  [71, 31, 20], [91, 13, 71, 31, 3, 21, 75, 91, 75, 86], [79, 14],
  [55, 21, 13, 71, 13, 32, 31], [71, 21, 79, 79, 61, 32], [14, 21, 79, 88],
  [71, 31, 75, 71], [23, 79, 33, 2, 71, 21, 73], [45]],
 [[92, 75, 88, 13, 86, 13, 20, 32], [79, 14], [32, 79, 86, 3, 13, 20, 21, 32],
  [83, 13, 86, 86, 20, 3], [13, 2], [71, 31, 20],
  [23, 79, 2, 14, 86, 13, 23, 71], [50, 79, 13, 2, 20, 3], [71, 31, 20],
  [61, 21, 79, 71, 20, 32, 71, 20, 21, 32], [91, 31, 79],
  [23, 75, 21, 21, 13, 20, 3], [96, 75, 2, 2, 20, 21, 32], [91, 13, 71, 31],
  [32, 33, 23, 31], [32, 86, 79, 30, 75, 2, 32], [75, 32], [69],
  [55, 33, 32, 31], [62, 33, 88, 96, 20, 21], [39, 2, 20],
  

In [ ]:
pprint(y_train_encoded[:2], compact=True)

[[13, 13, 13, 13, 13, 13, 10, 13, 13, 13, 13, 13, 10, 13, 13, 13, 13, 13, 7, 13,
  13, 13, 13, 13],
 [13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 6, 13,
  13, 13, 13, 13, 13, 13, 13, 13, 13, 13]]


## Data Loaders

In [ ]:
def pad_char(chars):
    batch_size = len(chars)
    max_seq_len = max(map(len, chars))
    pad_chars = [chars[idx] + [[0]] * (max_seq_len - len(chars[idx])) for idx in range(len(chars))]
    length_list = [list(map(len, pad_char)) for pad_char in pad_chars]
    max_word_len = max(map(max, length_list))
    char_seq_tensor = torch.zeros((batch_size, max_seq_len, max_word_len)).long()
    char_seq_lengths = torch.LongTensor(length_list)
    for idx, (seq, seqlen) in enumerate(zip(pad_chars, char_seq_lengths)):
        for idy, (word, wordlen) in enumerate(zip(seq, seqlen)):
            char_seq_tensor[idx, idy, :wordlen] = torch.LongTensor(word)

    return char_seq_tensor

In [ ]:
def custom_collate(batch):
    sent = [torch.tensor(item[0]) for item in batch]
    char_input = [item[1] for item in batch]
    tag = [torch.tensor(item[2]) for item in batch]
    lengths = torch.tensor([len(item[0]) for item in batch])

    padded_sent = nn.utils.rnn.pad_sequence(sent, batch_first=True, padding_value=PAD_ID)
    padded_tag = nn.utils.rnn.pad_sequence(tag, batch_first=True, padding_value=PAD_ID)
    padded_char = pad_char(char_input)

    batch = {"sent": padded_sent, "tag": padded_tag, "char": padded_char, "lengths": lengths}
    return batch

In [ ]:
train_dl = DataLoader(list(zip(X_train_encoded,X_train_char_encoded, y_train_encoded)), batch_size = 2, shuffle = False, collate_fn = custom_collate )

In [ ]:
example = next(iter(train_dl))

In [ ]:
example['sent'].shape, example['tag'].shape,  example['char'].shape, example['lengths'].shape

(torch.Size([2, 30]),
 torch.Size([2, 30]),
 torch.Size([2, 30, 13]),
 torch.Size([2]))

In [ ]:
example['sent']

tensor([[26712, 34197, 33274, 11587, 21753, 25622, 30270,  6375, 29484,  8734,
         34332,  4607, 15481, 18687, 26749,  8734, 22975, 34197, 29366, 31635,
         11812, 10530, 30933, 32151,     0,     0,     0,     0,     0,     0],
        [23800, 34197, 30830, 19378,  4607,  8734, 35019, 23610,  8734, 31310,
          2627, 25256, 34769,  8757,  9212,  9076, 16113, 17026,  6945, 13859,
         32377, 17675, 17026, 18687, 17026, 16654,  8734, 17523, 32151, 17026]])

In [ ]:
example['tag']

tensor([[13, 13, 13, 13, 13, 13, 10, 13, 13, 13, 13, 13, 10, 13, 13, 13, 13, 13,
          7, 13, 13, 13, 13, 13,  0,  0,  0,  0,  0,  0],
        [13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
          6, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]])

In [ ]:
example['char']

tensor([[[59, 31, 79, 33, 32, 75,  2,  3, 32,  0,  0,  0,  0],
         [79, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 3, 20, 88, 79,  2, 32, 71, 21, 75, 71, 79, 21, 32],
         [31, 75, 81, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [88, 75, 21, 23, 31, 20,  3,  0,  0,  0,  0,  0,  0],
         [71, 31, 21, 79, 33, 30, 31,  0,  0,  0,  0,  0,  0],
         [40, 79,  2,  3, 79,  2,  0,  0,  0,  0,  0,  0,  0],
         [71, 79,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 21, 79, 71, 20, 32, 71,  0,  0,  0,  0,  0,  0],
         [71, 31, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [91, 75, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [13,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [97, 21, 75, 48,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [75,  2,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 3, 20, 88, 75,  2,  3,  0,  0,  0,  0,  0,  0,  0],
         [71, 31, 20,  0,  0,  0,  0,  0,  0,  0,  0,  

In [ ]:
example['lengths']

tensor([24, 30])

In [ ]:
## dataloaders
batch_size = 32
train_dl = DataLoader(list(zip(X_train_encoded,X_train_char_encoded, y_train_encoded)), batch_size = batch_size, shuffle = True, collate_fn = custom_collate )
val_dl = DataLoader(list(zip(X_val_encoded,X_val_char_encoded, y_val_encoded)), batch_size = batch_size, shuffle = False, collate_fn = custom_collate )
test_dl = DataLoader(list(zip(X_test_encoded,X_test_char_encoded,y_test_encoded)), batch_size = batch_size, shuffle = False, collate_fn = custom_collate )

## Pretrained Vec

In [ ]:
## Pretrained Vectors
def load_pretrain_emb(filepath):
    lines = open(filepath,"r").readlines()
    embedd_dict = {}
    for line in lines:
        if len(line)>0:
            tokens = line.strip().split(" ")
            word = tokens[0]
            vec = tokens[1:]
            vec = np.array(vec).astype(float)
            embedd_dict[word]= vec

    return embedd_dict

def build_pretrain_embedding(filepath, vocab, emb_dim):
    embedd_dict = load_pretrain_emb(filepath)

    df_list = []

    for w,i in vocab.items():
        if w in embedd_dict:
            df_list.append(torch.tensor(embedd_dict[w]))
        elif w.lower() in embedd_dict:
            df_list.append(embedd_dict[w.lower()])
        else:
            random_vec = np.random.normal(size = (emb_dim))
            df_list.append(random_vec)


    return torch.tensor(df_list)



weights = build_pretrain_embedding("embeddings/glove.6B.100d.txt", word2idx, emb_dim=100)
weights.shape

/tmp/ipykernel_15595/624670142.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.tensor(df_list)


torch.Size([35180, 100])

## CharCNN

In [ ]:
class CharCNN(nn.Module):
    def __init__(self, alphabet_size, embedding_dim, hidden_dim, dropout):
        super(CharCNN, self).__init__()
        print("build char sequence feature extractor: CNN ...")
        self.hidden_dim = hidden_dim
        self.char_drop = nn.Dropout(dropout)
        self.char_embeddings = nn.Embedding(alphabet_size, embedding_dim, padding_idx= CHAR_PAD_ID)
        self.char_embeddings.weight.data.copy_(torch.from_numpy(CharCNN.random_embedding(alphabet_size, embedding_dim)))
        self.char_cnn = nn.Conv1d(embedding_dim, self.hidden_dim, kernel_size=3, padding=1)

    @staticmethod
    def random_embedding(vocab_size, embedding_dim):
        pretrain_emb = np.empty([vocab_size, embedding_dim])
        scale = np.sqrt(3.0 / embedding_dim)
        for index in range(vocab_size):
            pretrain_emb[index, :] = np.random.uniform(-scale, scale, [1, embedding_dim])
        pretrain_emb[0, :] = np.zeros((1, embedding_dim))
        return pretrain_emb

    def forward(self, input):
        batch_size = input.size(0)
        char_embeds = self.char_drop(self.char_embeddings(input))
        ## char_embeds: [batch size, num tokens, num_chars, embedding_dim]
        char_embeds = char_embeds[:,:,-1,:]   ## take embedding of last char
        ## char_embeds : [batch size, num tokens, embedding_dim]
        char_embeds = char_embeds.transpose(2,1)
         ## char_embeds : [batch size,  embedding_dim, num tokens,]
        char_cnn_out = self.char_cnn(char_embeds)
        # char_cnn_out: [batch size, hidden_dim, num tokens]
        # char_cnn_out = F.max_pool1d(char_cnn_out, char_cnn_out.size(2),padding = 0)
        # print(f"char_cnn_out : {char_cnn_out.shape}")
        return char_cnn_out.transpose(2,1)

## CRF

In [ ]:
START_TAG = -2
STOP_TAG = -1


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec, m_size):
    """
    calculate log of exp sum
    args:
        vec (batch_size, vanishing_dim, hidden_dim) : input tensor
        m_size : hidden_dim
    return:
        batch_size, hidden_dim
    """
    _, idx = torch.max(vec, 1)  # B * 1 * M
    max_score = torch.gather(vec, 1, idx.view(-1, 1, m_size)).view(-1, 1, m_size)  # B * M
    return max_score.view(-1, m_size) + torch.log(torch.sum(torch.exp(vec - max_score.expand_as(vec)), 1)).view(-1,
                                                                                                                m_size)  # B * M


class CRF(nn.Module):

    def __init__(self, tagset_size, gpu):
        super(CRF, self).__init__()
        print("build CRF...")
        self.gpu = gpu
        # Matrix of transition parameters.  Entry i,j is the score of transitioning from i to j.
        self.tagset_size = tagset_size
        # # We add 2 here, because of START_TAG and STOP_TAG
        # # transitions (f_tag_size, t_tag_size), transition value from f_tag to t_tag
        init_transitions = torch.zeros(self.tagset_size + 2, self.tagset_size + 2)
        init_transitions[:, START_TAG] = -10000.0
        init_transitions[STOP_TAG, :] = -10000.0
        init_transitions[:, 0] = -10000.0
        init_transitions[0, :] = -10000.0
        # if self.gpu:
        #     init_transitions = init_transitions.cuda()
        self.transitions = nn.Parameter(init_transitions)

        # self.transitions = nn.Parameter(torch.Tensor(self.tagset_size+2, self.tagset_size+2))
        # self.transitions.data.zero_()

    def _calculate_PZ(self, feats, mask):
        """
            input:
                feats: (batch, seq_len, self.tag_size+2)
                masks: (batch, seq_len)
        """
        batch_size = feats.size(0)
        seq_len = feats.size(1)
        tag_size = feats.size(2)
        # print feats.view(seq_len, tag_size)
        assert (tag_size == self.tagset_size + 2)
        mask = mask.transpose(1, 0).contiguous()
        ins_num = seq_len * batch_size
        ## be careful the view shape, it is .view(ins_num, 1, tag_size) but not .view(ins_num, tag_size, 1)
        feats = feats.transpose(1, 0).contiguous().view(ins_num, 1, tag_size).expand(ins_num, tag_size, tag_size)
        ## need to consider start
        scores = feats + self.transitions.view(1, tag_size, tag_size).expand(ins_num, tag_size, tag_size)
        scores = scores.view(seq_len, batch_size, tag_size, tag_size)
        # build iter
        seq_iter = enumerate(scores)
        _, inivalues = next(seq_iter)  # bat_size * from_target_size * to_target_size
        # only need start from start_tag
        partition = inivalues[:, START_TAG, :].clone().view(batch_size, tag_size, 1)  # bat_size * to_target_size

        ## add start score (from start to all tag, duplicate to batch_size)
        # partition = partition + self.transitions[START_TAG,:].view(1, tag_size, 1).expand(batch_size, tag_size, 1)
        # iter over last scores
        for idx, cur_values in seq_iter:
            # previous to_target is current from_target
            # partition: previous results log(exp(from_target)), #(batch_size * from_target)
            # cur_values: bat_size * from_target * to_target

            cur_values = cur_values + partition.contiguous().view(batch_size, tag_size, 1).expand(batch_size, tag_size,
                                                                                                  tag_size)
            cur_partition = log_sum_exp(cur_values, tag_size)
            # print cur_partition.data

            # (bat_size * from_target * to_target) -> (bat_size * to_target)
            # partition = utils.switch(partition, cur_partition, mask[idx].view(bat_size, 1).expand(bat_size, self.tagset_size)).view(bat_size, -1)
            mask_idx = mask[idx, :].view(batch_size, 1).expand(batch_size, tag_size)

            ## effective updated partition part, only keep the partition value of mask value = 1
            masked_cur_partition = cur_partition.masked_select(mask_idx)
            ## let mask_idx broadcastable, to disable warning
            mask_idx = mask_idx.contiguous().view(batch_size, tag_size, 1)

            ## replace the partition where the maskvalue=1, other partition value keeps the same
            partition.masked_scatter_(mask_idx, masked_cur_partition)
        # until the last state, add transition score for all partition (and do log_sum_exp) then select the value in STOP_TAG
        cur_values = self.transitions.view(1, tag_size, tag_size).expand(batch_size, tag_size,
                                                                         tag_size) + partition.contiguous().view(
            batch_size, tag_size, 1).expand(batch_size, tag_size, tag_size)
        cur_partition = log_sum_exp(cur_values, tag_size)
        final_partition = cur_partition[:, STOP_TAG]
        return final_partition.sum(), scores

    def _viterbi_decode(self, feats, mask):
        """
            input:
                feats: (batch, seq_len, self.tag_size+2)
                mask: (batch, seq_len)
            output:
                decode_idx: (batch, seq_len) decoded sequence
                path_score: (batch, 1) corresponding score for each sequence (to be implementated)
        """
        batch_size = feats.size(0)
        seq_len = feats.size(1)
        tag_size = feats.size(2)
        assert (tag_size == self.tagset_size + 2)
        ## calculate sentence length for each sentence
        length_mask = torch.sum(mask.long(), dim=1).view(batch_size, 1).long()
        ## mask to (seq_len, batch_size)
        mask = mask.transpose(1, 0).contiguous()
        ins_num = seq_len * batch_size
        ## be careful the view shape, it is .view(ins_num, 1, tag_size) but not .view(ins_num, tag_size, 1)
        feats = feats.transpose(1, 0).contiguous().view(ins_num, 1, tag_size).expand(ins_num, tag_size, tag_size)
        ## need to consider start
        scores = feats + self.transitions.view(1, tag_size, tag_size).expand(ins_num, tag_size, tag_size)
        scores = scores.view(seq_len, batch_size, tag_size, tag_size)

        # build iter
        seq_iter = enumerate(scores)
        ## record the position of best score
        back_points = list()
        partition_history = list()
        ##  reverse mask (bug for mask = 1- mask, use this as alternative choice)
        # mask = 1 + (-1)*mask
        mask = (1 - mask.long()).byte()
        _, inivalues = next(seq_iter)  # bat_size * from_target_size * to_target_size
        # only need start from start_tag
        partition = inivalues[:, START_TAG, :].clone().view(batch_size, tag_size)  # bat_size * to_target_size
        # print "init part:",partition.size()
        partition_history.append(partition)
        # iter over last scores
        for idx, cur_values in seq_iter:
            # previous to_target is current from_target
            # partition: previous results log(exp(from_target)), #(batch_size * from_target)
            # cur_values: batch_size * from_target * to_target
            cur_values = cur_values + partition.contiguous().view(batch_size, tag_size, 1).expand(batch_size, tag_size,
                                                                                                  tag_size)
            ## forscores, cur_bp = torch.max(cur_values[:,:-2,:], 1) # do not consider START_TAG/STOP_TAG
            # print "cur value:", cur_values.size()
            partition, cur_bp = torch.max(cur_values, 1)
            # print "partsize:",partition.size()
            # exit(0)
            # print partition
            # print cur_bp
            # print "one best, ",idx
            partition_history.append(partition)
            ## cur_bp: (batch_size, tag_size) max source score position in current tag
            ## set padded label as 0, which will be filtered in post processing
            cur_bp.masked_fill_(mask[idx].view(batch_size, 1).expand(batch_size, tag_size), 0)
            back_points.append(cur_bp)
        # exit(0)
        ### add score to final STOP_TAG
        partition_history = torch.cat(partition_history, 0).view(seq_len, batch_size, -1).transpose(1,
                                                                                                    0).contiguous()  ## (batch_size, seq_len. tag_size)
        ### get the last position for each setences, and select the last partitions using gather()
        last_position = length_mask.view(batch_size, 1, 1).expand(batch_size, 1, tag_size) - 1
        last_partition = torch.gather(partition_history, 1, last_position).view(batch_size, tag_size, 1)
        ### calculate the score from last partition to end state (and then select the STOP_TAG from it)
        last_values = last_partition.expand(batch_size, tag_size, tag_size) + self.transitions.view(1, tag_size,
                                                                                                    tag_size).expand(
            batch_size, tag_size, tag_size)
        _, last_bp = torch.max(last_values, 1)
        pad_zero = autograd.Variable(torch.zeros(batch_size, tag_size)).long()
        if self.gpu:
            pad_zero = pad_zero.cuda()
        back_points.append(pad_zero)
        back_points = torch.cat(back_points).view(seq_len, batch_size, tag_size)

        ## select end ids in STOP_TAG
        pointer = last_bp[:, STOP_TAG]
        insert_last = pointer.contiguous().view(batch_size, 1, 1).expand(batch_size, 1, tag_size)
        back_points = back_points.transpose(1, 0).contiguous()
        ## move the end ids(expand to tag_size) to the corresponding position of back_points to replace the 0 values
        # print "lp:",last_position
        # print "il:",insert_last
        back_points.scatter_(1, last_position, insert_last)
        # print "bp:",back_points
        # exit(0)
        back_points = back_points.transpose(1, 0).contiguous()
        ## decode from the end, padded position ids are 0, which will be filtered if following evaluation
        decode_idx = autograd.Variable(torch.LongTensor(seq_len, batch_size))
        if self.gpu:
            decode_idx = decode_idx.cuda()
        decode_idx[-1] = pointer.detach()
        for idx in range(len(back_points) - 2, -1, -1):
            pointer = torch.gather(back_points[idx], 1, pointer.contiguous().view(batch_size, 1))
            decode_idx[idx] = pointer.detach().view(batch_size)
        path_score = None
        decode_idx = decode_idx.transpose(1, 0)
        return path_score, decode_idx

    def _score_sentence(self, scores, mask, tags):
        """
            input:
                scores: variable (seq_len, batch, tag_size, tag_size)
                mask: (batch, seq_len)
                tags: tensor  (batch, seq_len)
            output:
                score: sum of score for gold sequences within whole batch
        """
        # Gives the score of a provided tag sequence
        batch_size = scores.size(1)
        seq_len = scores.size(0)
        tag_size = scores.size(2)
        ## convert tag value into a new format, recorded label bigram information to index
        new_tags = autograd.Variable(torch.LongTensor(batch_size, seq_len))
        if self.gpu:
            new_tags = new_tags.cuda()
        for idx in range(seq_len):
            if idx == 0:
                ## start -> first score
                new_tags[:, 0] = (tag_size - 2) * tag_size + tags[:, 0]

            else:
                new_tags[:, idx] = tags[:, idx - 1] * tag_size + tags[:, idx]

        ## transition for label to STOP_TAG
        end_transition = self.transitions[:, STOP_TAG].contiguous().view(1, tag_size).expand(batch_size, tag_size)
        ## length for batch,  last word position = length - 1
        length_mask = torch.sum(mask.long(), dim=1).view(batch_size, 1).long()
        ## index the label id of last word
        end_ids = torch.gather(tags, 1, length_mask - 1)

        ## index the transition score for end_id to STOP_TAG
        end_energy = torch.gather(end_transition, 1, end_ids)

        ## convert tag as (seq_len, batch_size, 1)
        new_tags = new_tags.transpose(1, 0).contiguous().view(seq_len, batch_size, 1)
        ### need convert tags id to search from 400 positions of scores
        tg_energy = torch.gather(scores.view(seq_len, batch_size, -1), 2, new_tags).view(seq_len,
                                                                                         batch_size)  # seq_len * bat_size
        ## mask transpose to (seq_len, batch_size)
        tg_energy = tg_energy.masked_select(mask.transpose(1, 0))

        # ## calculate the score from START_TAG to first label
        # start_transition = self.transitions[START_TAG,:].view(1, tag_size).expand(batch_size, tag_size)
        # start_energy = torch.gather(start_transition, 1, tags[0,:])

        ## add all score together
        # gold_score = start_energy.sum() + tg_energy.sum() + end_energy.sum()
        gold_score = tg_energy.sum() + end_energy.sum()
        return gold_score

    def neg_log_likelihood_loss(self, feats, mask, tags):
        # nonegative log likelihood
        batch_size = feats.size(0)
        forward_score, scores = self._calculate_PZ(feats, mask)
        gold_score = self._score_sentence(scores, mask, tags)
        # print "batch, f:", forward_score.data[0], " g:", gold_score.data[0], " dis:", forward_score.data[0] - gold_score.data[0]
        # exit(0)
        return forward_score - gold_score

## Build Model

In [ ]:
class NERModel(pl.LightningModule):
    """
    if you will you NLLLoss then you have to use log_softmax in forward else use CrossEntropy
    """
    def __init__(self, vocab_size, word_emb_dim, hidden_dim, n_tags, alphabet_size, char_embedding_dim, char_hidden_dim,
                learning_rate, dropout, bidirectional = False, n_layers = 1,
                 use_pretrained = True, use_crf = True, use_gpu = True, use_char = True):
        super().__init__()
        self.use_crf = use_crf
        self.crf = CRF(tagset_size= n_tags, gpu= use_gpu)
        self.use_char = use_char
        self.char_feature = CharCNN(alphabet_size, char_embedding_dim, char_hidden_dim, dropout)
        self.input_dim = word_emb_dim
        self.learning_rate = learning_rate
        self.bidirectional = bidirectional

        if self.use_char:
            self.input_dim += char_hidden_dim

        # metrics
        self.train_f1 = []
        self.val_f1 = []
        self.val_loss = []
        self.test_f1 =[]
        self.test_precision = []
        self.test_recall = []

        ## define loss
        if self.use_crf:
            self.loss_fn = self.crf.neg_log_likelihood_loss
        else:
            self.loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_ID)

        ## embedding layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim= word_emb_dim, padding_idx= PAD_ID)
        if use_pretrained:
            self.embedding.weight.data.copy_(weights)
        else:
            self.embedding.weight.data.copy_(torch.from_numpy(self.random_embedding(vocab_size, word_emb_dim)))

        ## lstm layer
        self.lstm = nn.LSTM(self.input_dim, hidden_dim, batch_first=True, bidirectional=bidirectional, dropout = dropout, num_layers = n_layers)

        ## last layer
        if self.use_crf:
            self.hidden2tag = nn.Linear(hidden_dim * 2, n_tags + 2)
        else:
            self.hidden2tag = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, n_tags)

        ## other layer
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def random_embedding(self, vocab_size, embedding_dim):
        pretrain_emb = np.empty([vocab_size, embedding_dim])
        scale = np.sqrt(3.0 / embedding_dim)
        for index in range(1, vocab_size):
            pretrain_emb[index, :] = np.random.uniform(-scale, scale, [1, embedding_dim])
        return pretrain_emb

    def forward(self, sent, char_inputs, lengths, verbose = False):

        ## layers
        word_embedding = self.embedding(sent)
        # print(f"word_embedding shape : {word_embedding.shape}")
        # word_embedding : [batch size, seq_len, emb dim]

        word_list = [word_embedding]
        if self.use_char:
            char_features = self.char_feature(char_inputs)
            # print(f"char_features shape : {char_features.shape}")
            word_list.append(char_features)
        embedded = torch.cat(word_list, 2)
        # print(f"final embedding shape : {embedded.shape}")


        packed_input = nn.utils.rnn.pack_padded_sequence(embedded, lengths.to('cpu'), batch_first = True, enforce_sorted = False)
        x, xlengths = nn.utils.rnn.pad_packed_sequence(packed_input, batch_first = True)

        output, (hidden, cell) = self.lstm(x)
        output = self.dropout(output)
        logits = self.hidden2tag(output)

        if self.use_crf:
            logits = logits
        else:
            logits = logits.permute(0,2,1)

        if verbose:
            print(f"Sent : {sent.shape}")
            print(f'length : {lengths.shape}')
            print(f'x : {x.shape}')
            print(f'xlengths : {xlengths.shape}')
            print(f'embedded : {embedded.shape}')
            print(f'output : {output.shape}')
            print(f'hidden : {hidden.shape}')
            print(f'cell : {cell.shape}')
            print(f'logits : {logits.shape}')

        return logits

    def calculate_metrics(self, y_true, y_pred, mask):

        y_true = y_true  * mask
        y_pred = y_pred * mask

        ## metrics
        y_true = y_true.cpu().numpy().tolist()
        y_pred = y_pred.cpu().numpy().tolist()
        y_true_label = [[idx2tag[tag] for tag in sent_tag] for sent_tag in y_true]
        y_pred_label = [[idx2tag[tag] for tag in sent_tag] for sent_tag in y_pred]

        f1_score = metrics.f1_score(y_true_label, y_pred_label)
        precision = metrics.precision_score(y_true_label, y_pred_label)
        recall = metrics.recall_score(y_true_label, y_pred_label)
        return f1_score, precision, recall

    def _shared_step(self, batch):
        sents, tags, char, lengths = batch['sent'], batch['tag'],batch['char'], batch['lengths']
        mask = (tags != PAD_ID)
        logits = self(sents, char, lengths)

        if self.use_crf:
            loss = self.loss_fn(logits, mask, tags)
            _, preds = self.crf._viterbi_decode(logits, mask)
        else:
            loss = self.loss_fn(logits, tags)
            _ , preds = torch.max(logits, dim = 1)

        ## calculate metrics
        f1_score, precision, recall = self.calculate_metrics(preds, tags, mask)
        return loss, f1_score, precision, recall

    def training_step(self, batch):
        loss, f1_score, precision, recall = self._shared_step(batch)
        self.train_f1.append(f1_score)
        self.log_dict({"train_loss": loss, "train_f1": np.mean(self.train_f1)}, on_step = False, on_epoch = True, prog_bar=  True)
        return loss

    def validation_step(self, batch):
        loss, f1_score, precision, recall = self._shared_step(batch)
        self.val_f1.append(f1_score)
        self.val_loss.append(loss.cpu().item())
        self.log_dict({"val_loss": loss, "val_f1": np.mean(self.val_f1)}, on_step = False, on_epoch = True, prog_bar=  True)
        return loss

    def on_training_epoch_end(self):
        self.train_f1 =[]

    def on_validation_epoch_end(self):
        print(f'Epoch : {self.current_epoch} Loss : {np.mean(self.val_loss)} F1 : {np.mean(self.val_f1)}')
        self.val_f1 =[]
        self.val_loss = []

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = self.learning_rate)
        return optimizer

    def test_step(self, batch, batch_idx):
        sents, tags, char, lengths = batch['sent'], batch['tag'], batch['char'], batch['lengths']
        mask = (tags != PAD_ID)
        logits = self(sents, char, lengths)

        if self.use_crf:
            _, preds = self.crf._viterbi_decode(logits, mask)
        else:
            _ , preds = torch.max(logits, dim = 1)

        ## calculate metrics
        f1_score, precision, recall = self.calculate_metrics(preds, tags, mask)
        self.test_f1.append(f1_score)
        self.test_precision.append(precision)
        self.test_recall.append(recall)

    def on_test_epoch_end(self):
        print(f'F1 : {np.mean(self.test_f1)} Precision : {np.mean(self.test_precision)} Recall : {np.mean(self.test_recall)}')
        self.test_f1 = []
        self.test_precision = []
        self.test_recall = []

In [ ]:
model= NERModel(vocab_size = len(word2idx),
                word_emb_dim = 100,
                hidden_dim = 64,
                n_tags = len(tag2idx),
                alphabet_size = len(char2idx),
                char_embedding_dim = 40,
                char_hidden_dim = 50,
                learning_rate = 1e-3,
                dropout = 0.3,
                bidirectional = True,
                n_layers = 2,
                use_pretrained= True,
                use_crf= True
                )


logits = model(example['sent'], example['char'], example['lengths'], verbose = True)
true_label = example['tag']
print(f"True label shape : {true_label.shape}")

model.crf.gpu = False
mask = (true_label != PAD_ID)
loss = model.loss_fn(logits, mask, true_label)
print(loss)

scores, tag_seq = model.crf._viterbi_decode(logits, mask)
print(tag_seq)

build CRF...
build char sequence feature extractor: CNN ...
Sent : torch.Size([2, 30])
length : torch.Size([2])
x : torch.Size([2, 30, 150])
xlengths : torch.Size([2])
embedded : torch.Size([2, 30, 150])
output : torch.Size([2, 30, 128])
hidden : torch.Size([4, 2, 64])
cell : torch.Size([4, 2, 64])
logits : torch.Size([2, 30, 20])
True label shape : torch.Size([2, 30])


tensor(152.6577, grad_fn=<SubBackward0>)
tensor([[10, 11, 10, 10, 10, 10, 10, 10, 10, 17, 11,  8, 10, 17, 11, 11, 11, 10,
         17, 11, 10, 10, 10, 10,  0,  0,  0,  0,  0, 10],
        [10, 11, 10, 17,  1, 17, 17, 11, 11, 11, 17, 10, 10, 10, 10, 10, 10, 17,
         17, 10, 17,  1, 10, 10, 17, 10, 10, 11, 11, 10]])


/tmp/ipykernel_15595/57801177.py:154: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:1646.)
  cur_bp.masked_fill_(mask[idx].view(batch_size, 1).expand(batch_size, tag_size), 0)


In [ ]:
## Model Training
model= NERModel(vocab_size = len(word2idx),
                word_emb_dim = 100,
                hidden_dim = 64,
                n_tags = len(tag2idx),
                alphabet_size = len(char2idx),
                char_embedding_dim = 40,
                char_hidden_dim = 50,
                learning_rate = 1e-3,
                dropout = 0.3,
                bidirectional = True,
                n_layers = 2,
                use_pretrained= True,
                use_crf= True,
                use_char=True
                )

callbacks = pl.callbacks.ModelCheckpoint(dirpath = "checkpoints_logs",
                                         filename = '{epoch}-{val_loss:.2f}-{val_f1:.2f}',
                                          mode = "min",
                                          monitor = "val_loss",
                                          save_last = True,
                                          save_top_k=-1)


trainer = pl.Trainer(accelerator= "gpu",
           max_epochs=5,
           check_val_every_n_epoch = 1,
           callbacks = [callbacks])

trainer.fit(model, train_dl, val_dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type      | Params
-------------------------------------------
0 | crf          | CRF       | 400   
1 | char_feature | CharCNN   | 10.1 K
2 | embedding    | Embedding | 3.5 M 
3 | lstm         | LSTM      | 209 K 
4 | hidden2tag   | Linear    | 2.6 K 
5 | relu         | ReLU      | 0     
6 | dropout      | Dropout   | 0     
-------------------------------------------
3.7 M     Trainable params
0         Non-trainable params
3.7 M     Total params
14.964    Total estimated model params size (MB)


build CRF...
build char sequence feature extractor: CNN ...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/tmp/ipykernel_15595/57801177.py:154: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/cuda/Indexing.cu:1435.)
  cur_bp.masked_fill_(mask[idx].view(batch_size, 1).expand(batch_size, tag_size), 0)


Epoch : 0 Loss : 1977.39111328125 F1 : 0.21277208848694418


...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch : 3 Loss : 55.05238498263889 F1 : 0.8783849786793029


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch : 4 Loss : 54.40173177083334 F1 : 0.8791902906857032


In [ ]:
## F1 : 0.8860281703349736 Precision : 0.8816257179809235 Recall : 0.89082280124986
trainer.test(model, dataloaders= test_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/saurabh/anaconda3/envs/lighting/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

/tmp/ipykernel_15595/57801177.py:154: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/cuda/Indexing.cu:1435.)
  cur_bp.masked_fill_(mask[idx].view(batch_size, 1).expand(batch_size, tag_size), 0)


F1 : 0.8860281703349736 Precision : 0.8816257179809235 Recall : 0.89082280124986


[{}]

## Predict

In [ ]:
model = model.eval()

In [ ]:
def process_data(text):
    text = text.strip().split(" ")
    lengths = len(text)
    word_encoded = []
    for w in text:
        word_encoded.append(word2idx.get(w, PAD_ID))

    text_tensor = torch.tensor(word_encoded).view(1, -1)
    lengths = torch.tensor([lengths])

    ## char
    char = to_char_number(text)
    char = [[torch.tensor(_id) for _id in _char_id] for _char_id in [char]]
    char_tensor = pad_char(char)

    return text_tensor, char_tensor, lengths


i = random.choices(list(range(len(test_sents))))[0]
text = test_sents[i]
true_label = test_tags[i].strip().split(" ")
text_tensor, char_tensor, lengths = process_data(text)
print(text_tensor.shape, char_tensor.shape, lengths.shape)

logits = model(text_tensor, char_tensor, lengths)
mask = torch.ones_like(text_tensor)

model.crf.gpu = False
_, preds = model.crf._viterbi_decode(logits, mask)

preds = preds.numpy()[0]
pred_labels = [idx2tag[p] for p in preds]

for w, p, t in zip(text.split(" "), pred_labels, true_label):
    print(f"{w:<20}  -->  {p:<5} --> {t:<5}")

torch.Size([1, 27]) torch.Size([1, 27, 11]) torch.Size([1])
Chinese               -->  B-gpe --> B-gpe
worker                -->  O     --> O    
sews                  -->  O     --> O    
clothing              -->  O     --> O    
at                    -->  O     --> O    
a                     -->  O     --> O    
garment               -->  O     --> O    
factory               -->  O     --> O    
in                    -->  O     --> O    
Beijing               -->  B-geo --> B-geo
China                 -->  I-geo --> I-geo
is                    -->  O     --> O    
criticizing           -->  O     --> O    
the                   -->  O     --> O    
European              -->  B-org --> B-org
Union                 -->  I-org --> I-org
's                    -->  O     --> O    
decision              -->  O     --> O    
to                    -->  O     --> O    
investigate           -->  O     --> O    
surging               -->  O     --> O    
imports               -->  O     --> 

/tmp/ipykernel_15595/57801177.py:154: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:1646.)
  cur_bp.masked_fill_(mask[idx].view(batch_size, 1).expand(batch_size, tag_size), 0)
